In [ ]:
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset


paws_dataset = load_dataset("paws", "labeled_final")

train_data = paws_dataset['train']
validation_data = paws_dataset['validation']
test_data = paws_dataset['test']


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 2.3 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer


model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')



modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import torch


In [ ]:
sentence_df1 = train_data['sentence1']
sentence_df2 = train_data['sentence2']

In [ ]:
em_1 = model.encode(sentence_df1[:4000])
em_2 = model.encode(sentence_df2[:4000])

In [ ]:
import numpy as np

In [ ]:
em_1.shape

(4000, 768)

In [ ]:
vm_1 = model.encode(validation_data['sentence1'][:1000])
vm_2 = model.encode(validation_data['sentence2'][:1000])

In [ ]:
tm_1 = model.encode(test_data['sentence1'][:1000])
tm_2 = model.encode(test_data['sentence2'][:1000])

In [ ]:
import keras
predictor = keras.models.Sequential([keras.layers.Dense(100,
                                                       input_shape=(1536,),
                                                       activation='relu'),
                                     keras.layers.Dense(100,
                                                       activation='relu'),
                                     keras.layers.Dense(1,
                                                       activation='sigmoid')])
predictor.compile(optimizer='adam',
                  loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
import numpy as np

In [ ]:
col_to_predict = np.ravel(train_data['label'][:4000])
validation_labels = np.ravel(validation_data['label'][:1000])
test_labels = np.ravel(test_data['label'][:1000])

In [ ]:
def concat(vector1, vector2):
    return np.concatenate((vector1, vector2), axis=1).reshape(-1, 1536)


In [ ]:
em_c = concat(em_1, em_2)
tm_c = concat(tm_1, tm_2)
vm_c = concat(vm_1,vm_2)
print(len(em_c), len(tm_c), len(vm_c), len(col_to_predict), len(validation_labels), len(test_labels))

4000 1000 1000 4000 1000 1000


In [ ]:
predictor.fit(em_c, col_to_predict, validation_data=(vm_c, validation_labels), epochs=100)



Epoch 1/100
125/125 [==============================] - 1s 8ms/step - loss: 0.7850 - accuracy: 0.5295 - val_loss: 0.6599 - val_accuracy: 0.6130
Epoch 2/100
125/125 [==============================] - 1s 9ms/step - loss: 0.6754 - accuracy: 0.5795 - val_loss: 0.6383 - val_accuracy: 0.6360
Epoch 3/100
125/125 [==============================] - 1s 8ms/step - loss: 0.6639 - accuracy: 0.5938 - val_loss: 0.6334 - val_accuracy: 0.6340
Epoch 4/100
125/125 [==============================] - 1s 5ms/step - loss: 0.6472 - accuracy: 0.6205 - val_loss: 0.6404 - val_accuracy: 0.6340
Epoch 5/100
125/125 [==============================] - 1s 6ms/step - loss: 0.6273 - accuracy: 0.6450 - val_loss: 0.6220 - val_accuracy: 0.6540
Epoch 6/100
125/125 [==============================] - 1s 5ms/step - loss: 0.5996 - accuracy: 0.6733 - val_loss: 0.6036 - val_accuracy: 0.6620
Epoch 7/100
125/125 [==============================] - 1s 5ms/step - loss: 0.5692 - accuracy: 0.6990 - val_loss: 0.6249 - val_accuracy: 0.6550

In [ ]:
predictor.evaluate(tm_c, test_labels, verbose = 1)

32/32 [==============================] - 0s 2ms/step - loss: 2.2963 - accuracy: 0.5180


[2.296260118484497, 0.5180000066757202]

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
em_1_tensors = [torch.tensor(arr) for arr in em_1]
em_2_tensors = [torch.tensor(arr) for arr in em_2]


cos_c_similarities = []


for vec1, vec2 in zip(em_1_tensors, em_2_tensors):

    vec1 = vec1.squeeze()  # Remove the first dimension
    vec2 = vec2.squeeze()  # Remove the first dimension


    cosine_sim = F.cosine_similarity(vec1, vec2, dim=0)


    cos_c_similarities.append(cosine_sim.item())

print(len(cos_c_similarities))

4000


In [ ]:
tm_1_tensors = [torch.tensor(arr) for arr in tm_1]
tm_2_tensors = [torch.tensor(arr) for arr in tm_2]


cos_similarities = []


for vec1, vec2 in zip(tm_1_tensors, tm_2_tensors):

    vec1 = vec1.squeeze()  # Remove the first dimension
    vec2 = vec2.squeeze()  # Remove the first dimension


    cosine_sim = F.cosine_similarity(vec1, vec2, dim=0)


    cos_similarities.append(cosine_sim.item())

print(len(cos_similarities))

1000


In [ ]:
t_l = train_data['label']
cnt = 0
for i in range(len(t_l)):
    if(t_l[i]==1):
        cnt+=1
cnt

21829

In [ ]:
per = 0
ccc = cos_c_similarities.copy()
s = sum(cos_c_similarities)/len(cos_c_similarities)
for i in range(len(ccc)):
    if(cos_c_similarities[i]>s):
        ccc[i] = 1
    else:
        ccc[i] = 0
o_c = 0


for i in range(len(ccc)):
    if(ccc[i]==t_l[i]):
        per+=1

print(per)

2184


In [ ]:
te_l = test_data['label']
cnt = 0
for i in range(len(te_l)):
    if(te_l[i]==1):
        cnt+=1
cnt

3536

In [ ]:
per = 0
ccc = cos_similarities.copy()

for i in range(len(ccc)):
    if(cos_similarities[i]>s):
        ccc[i] = 1
    else:
        ccc[i] = 0
o_c = 0


for i in range(len(ccc)):
    if(ccc[i]==te_l[i]):
        per+=1

print(per)

560


In [ ]:
# CITES FOR TASK - 2

In [ ]:
# cites: BERT Embeddings: https://discuss.huggingface.co/t/how-to-get-embedding-matrix-of-bert-in-hugging-face/10261
# cites: Numpy: documentation + https://www.youtube.com/watch?v=QUT1VHiLmmI
# cites: Keras: https://keras.io/examples/nlp/semantic_similarity_with_bert/
# other cites: In the report.
# Doc2Vec: https://www.kaggle.com/code/petebleackley/doc2vec-and-keras/notebook